<a href="https://colab.research.google.com/github/jaunty0216/SomeAttempts/blob/main/%E6%96%87%E5%AD%97%E9%9B%B2%E8%A3%BD%E9%80%A0%E5%AE%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jieba

In [48]:
from google.colab import files
uploaded = files.upload()

Saving map_comments.txt to map_comments.txt


In [66]:
def Paintit():
  for filename,_ in uploaded.items():
    filepath = f"/content/{filename}"
    #print(f"Filepath: {filepath}")

  import pandas as pd
  OriginFile = pd.read_table(filepath,header=None,names=['contents'])
  #這邊遇到的問題：我的第一行會自動變成標頭，"header=None"解決了這問題
  #但標頭會變成'0'，"names=['contents']"解決了這問題
  Origin_df = pd.DataFrame(OriginFile)
  print(Origin_df)

  import jieba
  for _, content in Origin_df.iterrows():
    word = content['contents']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [64]:
Paintit()

                         contents
0             這邊想跟大家推薦老夫婦開的師大鹽酥雞，
1         以及師大的沙威瑪，因為個人最近澱粉攝取過量導致
2        身體健康亮紅燈，但我又非常想吃炸雞配沙威瑪，結果
3    兩家店共同想到這種方法----拿炸過的青椒代替沙威瑪餅皮
4      ，我這樣吃一個月了，當然也歡迎大家一起來這樣試吃看看
5  ，把他們吃爆（因為訂製，該產品為$200元，但真的比較健康）
